**Q1. Running Elastic**

In [ ]:
docker start $(docker ps -a -q --filter "status=exited")
aee3153dfa12
07e1b35812da

In [ ]:
docker start aee3153dfa12

curl localhost:9200

{
  "name" : "d46b546fc702",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "xsjMZdEHQK2nq--75P-oTg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    *"build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",*
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}

**Getting the data**

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

**Q2. Indexing the data**

Which function do you use for adding your data to elastic?
index

**Q3. Searching**

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': 'aee3153dfa12', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'aK41VxYZSI2Slxzgsk4Tjg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es_client.indices.create(index=index_name, body=index_settings)

response

In [5]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },

            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    return response

In [6]:
query = "How do I execute a command in a running docker container?"

In [7]:
elastic_search_response = elastic_search(query)

In [8]:
elastic_search_response.keys()

dict_keys(['took', 'timed_out', '_shards', 'hits'])

In [9]:
elastic_search_response['hits'].keys()

dict_keys(['total', 'max_score', 'hits'])

In [10]:
elastic_search_response['hits']['max_score']

84.050095

**Q4. Filtering**

In [13]:
def elastic_search(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    results_doc = []
    for hit in response['hits']['hits']:
        results_doc.append(hit['_source'])
    return response, results_doc

In [14]:
response_4, results_doc_4 = elastic_search(query)
for hit in response_4['hits']['hits']:
    print(hit['_source']['question'])

How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


**Q5. Building a prompt**

In [34]:
question = "How do I execute a command in a running docker container?"

In [60]:
context_template = """
Q: {question}
A: {text}
""".strip()

context_result = ""

for doc in results_doc_4:
    doc_str = context_template.format(**doc)
    context_result += ("\n\n" + doc_str)

context = context_result.strip()
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [62]:
prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [63]:
prompt = prompt_template.format(question=question, context=context_template).strip()
len(prompt)

1462

**Q6. Tokens**

In [22]:
import tiktoken

In [64]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [66]:
len(encoding.encode(prompt))

322

**Bonus: generating the answer (ungraded)**

In [25]:
from openai import OpenAI

In [26]:
client = OpenAI(api_key="")

In [27]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    results_doc = []
    for hit in response['hits']['hits']:
        results_doc.append(hit['_source'])
    return results_doc

In [28]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user", "content":prompt}]
    )
    
    return response.choices[0].message.content

In [29]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [30]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [31]:
rag(query)

'To execute a command in a running Docker container, you can use the `docker exec` command. Here are the steps to do it:\n\n1. List the running containers to find the container ID or name:\n   ```sh\n   docker ps\n   ```\n\n2. Once you have the container ID or name, you can execute a command in that container. For example, to open a bash shell in the container, use the following command:\n   ```sh\n   docker exec -it <container-id> bash\n   ```\n\nReplace `<container-id>` with the actual ID of the container you wish to execute the command in.'